# MATH 693A Advanced Numerical Methods: Computational Optimization HW 3
### By Will McGrath

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem 1
### Write a Program that implements the dogleg method. Choose $B_k$ to be the exact Hessian. Apply it to solve Rosenbrock’s function: $f(x) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2$. 
### Use an initial trust region radius of 1. Set maximum trust region radius to 300. Use the initial point: $x_0 = [-1.2, 1]$ and then try another point $x_0 = [2.8, 4]$. Do the following for each of the initial points.

### Here are the parameters you should use for the Dogleg Algorithm:
### a. Use $\| \nabla{f(x_k)} \|< 10^{-8}$ as the stopping criteria for your optimization algorithm.
### b. State the total number of iterations obtained in your optimization algorithm.
### c. Plot the objective function $f(x)$. On the same figure, plot the $x_k$ values at the different iterates of your optimization algorithm.
### d. Plot the size of the objective function as a function of the iteration number. Use semi-log plot.
### e. You should hand in (i) your code (ii) the first 4 and last 4 values of $x_k$ obtained from your program.
### f. Determine the minimizer of the Rosenbrock function $x^*$.


### Trust-region method uses a quadratic model. At each iteration, the step is calculated by solving the following quadratic problem (sub-problem) using dogleg method:
> Note: radius increases only if $|p_k|$ reaches the trust-region border
- ## $\bar{p_k} = \argmin_{\| \bar{p} \| \le \Delta{_k}}[f(\bar{x_k}) + \bar{p}^T\nabla{f(\bar{x_k})} + \frac{1}{2}\bar{p}^TB_k\bar{p}]$
    - ### When the first three terms of the quadratic model agrees with the Taylor expansion: S.T. $B_k = \nabla^2f(\bar{x_k})$, the algorithm is called the trust-region Newton Method
### The locally constrained trust region problem is to minimize model $m_k$ whhich is based on the Taylor expansion of the objective $f$ at the current point(where $T_k$ is the trust region and $\bar{p_k}$ is now $\bar{p}$ since $\bar{m_k}$ is being iterated):
- ## $ \bar{p_k} = \min_{\bar{p} \in T_k} m_k(\bar{p}) = \min_{\bar{p} \in T_k} [f(\bar{x_k}) + \bar{p}^T\nabla{f(\bar{x_k})} + \frac{1}{2}\bar{p}^TB_k\bar{p}]$
### The full step is the unconstrained minimum of the quadratic model: 
- ## $\bar{p_k}^{FS} = -B_k^{-1}\nabla{f(\bar{x_k})}$
### The step in the steepest descent direction is given by the unconstrained minimum of the quadratic model along the steepest descent direction:
- ## $\bar{p_k}^{U} = - \frac{\nabla{f(\bar{x_k})}^T\nabla{f(\bar{x_k})}}  {\nabla{f(\bar{x_k})}^T B_k \nabla{f(\bar{x_k})}} \nabla{f(\bar{x_k})}$

In [ ]:
# Rosenbrock function
def objective_func(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]

    return 100*(y - x**2)**2 + (1 - x)**2

def gradient(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]

    return np.array([400*x**3 - 400*x*y + 2*x - 2, 200*(y - x**2)])

def hessian(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]
    
    return np.array([[1200*x**2 - 400*y + 2, -400*x],[-400*x, 200]])

### Dogleg Method:

In [ ]:
def dogleg_method(grad, Hk, Bk, trust_region):
    pass

### Trust Region: 
- ### Set $k = 1, \hat{\Delta} > 0, \Delta_0 \in (0, \hat{\Delta}),$ and $\eta \in (0, \frac{1}{4})$ S.T. $\hat{\Delta}$ = max trust region radius, and $\Delta_0$ = initial trust region radius
- ### Given a step $\bar{p_k}$ we define the ratio: $\rho_k=\frac{actual \ reduction}{predicted \ reduction} = \frac{f(\bar{x_k}) \ - \ f(\bar{x_k} + \bar{p_k})}{m_k(0) \ - \ m_k(\bar{p_k})}$
- ### If $\rho_k < 0$ We shrink the size of the trust region.
- ### If $\rho_k ≈ 0$ Then we shrink the size of the trust region.
- ### If $\rho_k ≈ 1$ Then the model is in good agreement with the objective; in this case it is (probably) safe to expand the trust region for the next iteration.
- ### Else we keep the size of the trust region.

In [ ]:
# trust region algorithm 
def trust_reg_dogleg(x0, obj_func, grad, hess, grad_stop_criteria, eta=0.15, initial_trust_radius=1, max_trust_radius=300):
    xbar_k = x0 # xbar_k = xbar_transposed
    trust_reg = initial_trust_radius
    k = 1

    while np.linalg.norm(grad(xbar_k)) > grad_stop_criteria:
        Bk = hess(xbar_k)
        Hk = np.linalg.inv(Bk) # hessian of Bk inverse

        # get approx. step pbar_k by dogleg method (gives minimized pbar_k)
        pbar_k = dogleg_method(grad, Hk, Bk, trust_reg)

        print(pbar_k)

        # define a ratio measuring the success of a step
        # given a step pbar_k we define the ratio: rho_k = actual reduction / predicted reduction 
        mk_0 = obj_func(xbar_k)
        mk_pbar_k = obj_func(xbar_k) + np.dot(grad(xbar_k),  pbar_k) + 0.5 * np.dot(pbar_k, np.dot(Bk, pbar_k))
        act_reduc = obj_func(xbar_k) - obj_func(xbar_k + pbar_k)
        pred_reduc = mk_0 - mk_pbar_k
        rho_k = act_reduc / pred_reduc

        # rho is close to zero or negative, therefore the trust region must shrink
        if rho_k < 0.25:
            trust_reg = 0.25 * trust_reg

        # rho is close to one and pbar_k has reached the boundary of the trust region, therefore the trust region must be expanded
        # euclidean norm of pbar_k = sqrt(np.dot(pbar_k, pbar_k)) = np.linalg.norm(pbar_k)
        else:
            if rho_k > 0.75 and np.linalg.norm(pbar_k) == trust_reg:
                trust_reg = min(2 * trust_reg, max_trust_radius)
            else:
                trust_reg = trust_reg

        # choose position for the next iteration
        if rho_k > eta:
            xbar_k = xbar_k + pbar_k
        else:
            xbar_k = xbar_k
        
        k = k + 1
        
    return xbar_k

In [ ]:
res = trust_reg_dogleg([-1.2, 1], objective_func, gradient, hessian, 10**(-8))
print("Result of trust region dogleg method: {}".format(res))
print("Value of function at a point: {}".format(objective_func(res)))

In [ ]:
objective_func([-1.2, 1])

# Problem 2
### Experiment with the update rule for the trust region by changing the constants in Algorithm 4.1 in the text Numerical Optimization by Nocedal and Wright 2006. State what you experimented with and discuss your observations.

In [ ]:
def jumpingOnClouds(c):
    # Write your code here
    num_of_jumps = 0
    indx = 0
    while indx < len(c)-1:
        if indx+2 < len(c) and c[indx+2] == 0:
            indx += 2
            num_of_jumps +=1

        else:
            indx += 1
            num_of_jumps += 1

    return num_of_jumps

In [ ]:
c = [0,0,1,0,0,0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
#c = [0,0]
jumpingOnClouds(c)